**Table of contents**<a id='toc0_'></a>    
- [Self-Localization Using Particle Filter](#toc1_)    
- [Launching pybullet](#toc2_)    
- [Initial Settings](#toc3_)    
- [define Odometry class](#toc4_)    
- [Define Particle Filter Class](#toc5_)    
- [Setting Parameters for Mobile Robots](#toc6_)    
- [Setting Parameters for Sensors](#toc7_)    
- [Setting Up the Environment](#toc8_)    
- [Setting Parameters for Particles](#toc9_)    
- [Running the Simulation](#toc10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Self-Localization Using Particle Filter](#toc0_)
In this notebook, we will perform self-localization using a particle filter for a mobile robot.

(For a manual summarizing the functions available in pybullet, please refer to [here](https://github.com/bulletphysics/bullet3/blob/master/docs/pybullet_quickstartguide.pdf).)


# <a id='toc2_'></a>[Launching pybullet](#toc0_)

In [1]:
import time
import math
import pybullet
import pybullet_data
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
physics_client = pybullet.connect(pybullet.GUI) 

pybullet build time: Nov 28 2023 23:45:17


startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa
GL_RENDERER=llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.2
Vendor = Mesa
Renderer = llvmpipe (LLVM 15.0.7, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started



# <a id='toc3_'></a>[Initial Settings](#toc0_)


In [2]:
import random

pybullet.resetSimulation() # Reset the simulation space
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath()) # Add path to necessary data for pybullet
pybullet.setGravity(0.0, 0.0, -9.8) # Set gravity to that of Earth
time_step = 1./240.
# time_step = 0.1
pybullet.setTimeStep(time_step)

# Load the floor
plane_id = pybullet.loadURDF("plane.urdf")

ven = Mesa
ven = Mesa


# <a id='toc4_'></a>[define Odometry class](#toc0_)

In [3]:
class WheelOdometry:
    def __init__(self, initial_pose, wheel_radius, wheel_tread, const_right_wheel, const_left_wheel, initial_angle_right_wheel, initial_angle_left_wheel):
        """
        Class for wheel odometry
        Parameters
        ----------
        initial_pose : tuple
            Initial position and orientation of the robot (x, y, theta)
        wheel_radius : float
            Radius of the wheel [m]
        wheel_tread : float
            Distance between the left and right wheels [m]
        const_right_wheel : float
            Constant for the right wheel (correction value due to friction and environment)
        const_left_wheel : float
            Constant for the left wheel (correction value due to friction and environment)
        initial_angle_right_wheel : float
            Initial angle of the right wheel [rad]
        initial_angle_left_wheel : float
            Initial angle of the left wheel [rad]
        """
        # Constants related to the size of the wheels
        self.WHEEL_DIAMETER = wheel_radius * 2
        self.WHEEL_TREAD = wheel_tread

        # Constants considering the environment and tire material
        self.CONSTANT_RIGHT_WHEEL = const_right_wheel
        self.CONSTANT_LEFT_WHEEL = const_left_wheel

        # Distance per revolution
        self.ONE_REVOLUTION_DISTANCE_RIGHT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_LEFT_WHEEL
        self.ONE_REVOLUTION_DISTANCE_LEFT = math.pi * self.WHEEL_DIAMETER * self.CONSTANT_RIGHT_WHEEL

        # Initial position and orientation of the robot (x, y, theta)
        self.x = initial_pose[0]
        self.y = initial_pose[1]
        self.theta = initial_pose[2]

        # Previous wheel angles
        self.last_angle_right_wheel = initial_angle_right_wheel
        self.last_angle_left_wheel = initial_angle_left_wheel

    def update_position(self, current_angle_right_wheel, current_angle_left_wheel):
        """
        Update the position and orientation of the robot
        
        Parameters
        ----------
        current_angle_right_wheel : float
            Current angle of the right wheel [rad]
        current_angle_left_wheel : float
            Current angle of the left wheel [rad]
        """
        # Calculate the small change in rotation angle [ΔΘ] of the left and right wheels
        delta_angle_right_wheel = current_angle_right_wheel - self.last_angle_right_wheel
        delta_angle_left_wheel = current_angle_left_wheel - self.last_angle_left_wheel

        # Calculate the number of revolutions from the small change in rotation angle [Δθ] of the left and right wheels (convert 2π → 1 revolution)
        revolution_right_wheel = delta_angle_right_wheel / (2.0 * math.pi)
        revolution_left_wheel = delta_angle_left_wheel / (2.0 * math.pi)

        # Calculate the distance traveled [m] by the left and right wheels
        distance_right_wheel = revolution_right_wheel * self.ONE_REVOLUTION_DISTANCE_RIGHT
        distance_left_wheel = revolution_left_wheel * self.ONE_REVOLUTION_DISTANCE_LEFT

        # Calculate the average travel distance [m] from the left and right travel distances
        distance_avg = (distance_right_wheel + distance_left_wheel) / 2.0

        # Update the position and orientation of the robot
        self.x += distance_avg * math.cos(self.theta)
        self.y += distance_avg * math.sin(self.theta)
        self.theta += math.atan2(distance_right_wheel - distance_left_wheel, self.WHEEL_TREAD)

        # Keep theta within the range -π to π
        if self.theta > math.pi:
            self.theta -= 2 * math.pi
        elif self.theta < -math.pi:
            self.theta += 2 * math.pi

        # Save the current wheel angles
        self.last_angle_right_wheel = current_angle_right_wheel
        self.last_angle_left_wheel = current_angle_left_wheel

    def get_position(self):
        """
        Get the position and orientation of the robot
        """
        return self.x, self.y, self.theta

# <a id='toc5_'></a>[Define Particle Filter Class](#toc0_)
We will define the Particle Filter class.

In self-localization of a mobile robot, particles in a particle filter are like **virtual replicas** of the actual robot, used to estimate the robot's position and orientation.

**Since they are replicas, they are treated as if they can perform odometry calculations and obtain sensor values just like the actual robot**.

However, since particles are virtual replicas, they cannot perform odometry calculations or obtain sensor values in the same way as the actual robot. Therefore, we perform pseudo-odometry calculations and obtain pseudo-sensor values to make estimations. (For details, refer to the explanations of each method within the class.)


The overall flow is as follows:
1. Initialization of particles: Scatter each particle randomly.
    - Implemented in the `__init__(num_particles)` method.
2. Update particle positions: Update the positions of the particles based on the "motion model" (equivalent to the odometry of a real robot).
    - Implemented in the `particle_filter.update_motion_model(v, omega, time_step)` method.
3. Update particle sensor values: Obtain the pseudo sensor values mounted on the particles.
    - Implemented in the `particle_filter.measurement_sensor()` method.
4. Update particle weights: Compare the "sensor values of the particles" with the "sensor values of the real robot" and update the weights so that particles with smaller errors have larger weights.
    - Implemented in the `particle_filter.update_weight(obstacle_distances)` method.
5. Resampling: Resample the particles based on their weights. This causes particles to gather at positions with higher weights, i.e., more accurate positions.
    - Implemented in the `particle_filter.resampling()` method.
6. Estimate particle positions: Estimate the position of the real robot by taking the weighted average of the resampled particle positions.
    - Implemented in the `particle_filter.estimate_pose()` method.


In [4]:
import copy

class ParticleFilter():
    class Particle:
        def __init__(self, x: float, y: float, theta: float, num_particles: int):
            """
            Initialize a particle
            Parameters
            ---
            x : float
                Particle position x
            y : float
                Particle position y
            theta : float
                Particle orientation θ
            num_particles : int
                Number of particles
            """
            # Particle position and orientation
            self.x = x
            self.y = y
            self.theta = theta

            # List to store distances between the particle and each obstacle
            self.obstacle_distances = []

            # Particle weight
            self.weight = 1. / num_particles

    def __init__(self, init_pos: list, 
                 num_particles: int, particle_range_x: list, particle_range_y: list, particle_range_theta: list,
                 v_noise_dist: float, omega_noise_dist: float, sensor_noise_dist: float, obstacle_positions: list,
                 resampling_x_noise_dist: float, resampling_y_noise_dist: float, resampling_theta_noise_dist: float,
                 draw_particles=False, particle_size=0.02):
        """
        Initialize the particle filter
        Parameters
        ---
        init_pos : list
            Initial position of the robot [x, y, theta]
        num_particles : int
            Total number of particles
        particle_range_x : list
            Range of initial particle positions x [min, max]
        particle_range_y : list
            Range of initial particle positions y [min, max]
        particle_range_theta : list
            Range of initial particle orientations θ [min, max]
        v_noise_dist : float
            Variance of velocity noise
        omega_noise_dist : float
            Variance of angular velocity noise
        sensor_noise_dist : float
            Variance of sensor noise
        resampling_x_noise_dist : float
            Variance of x-direction noise during resampling
        resampling_y_noise_dist : float
            Variance of y-direction noise during resampling
        resampling_theta_noise_dist : float
            Variance of θ-direction noise during resampling
        obstacle_positions : list
            Positions of obstacles [[x1, y1], [x2, y2], ...] (assuming obstacles are static and do not move)
        draw_particles : bool
            Whether to draw particles
        particle_size : float
            Size of particles (effective only if draw_particles is True)
        """
        # Estimated position by particle filter
        self.x = init_pos[0]
        self.y = init_pos[1]
        self.theta = init_pos[2]

        # Settings related to particles
        self.num_particles = num_particles
        self.v_dist = v_noise_dist
        self.omega_dist = omega_noise_dist
        self.sensor_noise_dist = sensor_noise_dist
        self.resampling_x_noise_dist = resampling_x_noise_dist
        self.resampling_y_noise_dist = resampling_y_noise_dist
        self.resampling_theta_noise_dist = resampling_theta_noise_dist

        # Settings related to obstacles
        self.obstacle_positions = obstacle_positions

        # Settings for drawing particles
        self.draw_particles = draw_particles

        # Randomly generate initial positions and orientations of particles
        self.particles = []
        self.particle_ids = [] # For drawing
        for i in range(self.num_particles):
            x_init = random.uniform(particle_range_x[0], particle_range_x[1])
            y_init = random.uniform(particle_range_y[0], particle_range_y[1])
            theta_init = random.uniform(particle_range_theta[0], particle_range_theta[1])
            particle = self.Particle(x_init, y_init, theta_init, self.num_particles)
            self.particles.append(particle)

            # For drawing
            if self.draw_particles:
                # Create visual for the particle
                particle_visual_id = pybullet.createVisualShape(pybullet.GEOM_SPHERE, radius=particle_size, rgbaColor=[0,1,0,1])
                particle_id = pybullet.createMultiBody(0, -1, particle_visual_id, [self.particles[i].x, self.particles[i].y, 0.2], useMaximalCoordinates=True)
                self.particle_ids.append(particle_id)

    def update_motion_model(self, v: float, omega: float, dt: float):
        """
        Update the position of particles
            Corresponds to position update by odometry in the actual robot
            However, particles are virtual replicas of the actual robot and cannot obtain odometry information using encoders,
            so the position is updated based on the motion model using the "velocity" and "angular velocity" obtained from the actual robot
            Also, since the result of odometry in the actual robot is affected by friction and other factors, errors are added when updating the position of particles
        Parameters
        ---
        v : float
            Velocity of the actual robot
        omega : float
            Angular velocity of the actual robot
        dt : float
            Simulation time step
        """
        for i in range(self.num_particles):
            # Generate noise for velocity and angular velocity
            v_noise = random.gauss(0, self.v_dist)
            omega_noise = random.gauss(0, self.omega_dist)

            # Update the position of the particle
            self.particles[i].x += (v + v_noise) * math.cos(self.particles[i].theta) * dt
            self.particles[i].y += (v + v_noise) * math.sin(self.particles[i].theta) * dt
            self.particles[i].theta += (omega + omega_noise) * dt

            # Keep theta within the range -π to π
            if self.particles[i].theta > math.pi:
                self.particles[i].theta -= 2 * math.pi
            elif self.particles[i].theta < -math.pi:
                self.particles[i].theta += 2 * math.pi

            # For drawing
            if self.draw_particles:
                pybullet.resetBasePositionAndOrientation(self.particle_ids[i], [self.particles[i].x, self.particles[i].y, 0.15], pybullet.getQuaternionFromEuler([0, 0, self.particles[i].theta]))


    def measurement_sensor(self):
        """
        Measure the distance to obstacles for each particle (observed value)
            Obtain sensor values from the sensors of the particles
            However, particles are virtual replicas of the actual robot and do not actually have sensors, so measurements are made using pseudo-sensors
            In this case, since the positions of the obstacles are known, the distance between the "current position of the particle" and "each obstacle" is treated as the sensor value
            Also, since the sensor values of the actual robot are affected by noise, noise is added to the pseudo-sensor values of the particles

            (Supplement) If the actual robot uses a LiDAR sensor:
                Each particle is also given a pseudo-LiDAR sensor to measure the distance to obstacles.
                Of course, particles do not actually have LiDAR sensors, so it is necessary to pseudo-reproduce LiDAR
                Specifically, prepare an environment map and measure the distance based on that map information
                For example, perform ray casting on the map to measure the distance from the "coordinate position of the particle" to the "coordinate position of the black pixel (obstacle)".
                (Note that since measurements are made on the image, it is necessary to convert the measurement results to actual distances. Therefore, it is necessary to prepare an environment map that can convert distances)
        """
        for i in range(self.num_particles):
            # Initialize distances to each obstacle
            self.particles[i].obstacle_distances = []

            # Measure the distance to all obstacles
            for obstacle_position in self.obstacle_positions:
                # Measure the distance to the obstacle
                distance = math.sqrt((self.particles[i].x - obstacle_position[0])**2 + (self.particles[i].y - obstacle_position[1])**2)
                distance += random.gauss(0, self.sensor_noise_dist) # Add sensor noise
                self.particles[i].obstacle_distances.append(distance)
            
    def update_weight(self, obstacle_distance_real_robot: list):
        """
        Update the weight of particles
            For each particle, calculate the error between the "sensor value of the actual robot" and the "sensor value of the particle (pseudo)", and calculate the weight based on that error
            More specifically, if the error is small, it can be predicted that the position of the particle is close to the position of the actual robot, so the weight is increased
            Conversely, if the error is large, it can be predicted that the position of the particle is far from the position of the actual robot, so the weight is decreased
        Parameters
        ---
        obstacle_distance_real_robot : list
            Measurement results of the actual robot's sensor [[distance of the first ray], [distance of the second ray], ...]
        """
        for i in range(self.num_particles):
            for j in range(len(obstacle_distance_real_robot)):
                error = abs((obstacle_distance_real_robot[j] - self.particles[i].obstacle_distances[j])) # Calculate the total error between the "distance measured by the actual robot's sensor" and the "distance measured by the particle's sensor"
                self.particles[i].weight *= self.gaussian_probability(error, math.sqrt(self.sensor_noise_dist)) # Calculate the weight based on the Gaussian distribution from the error (the smaller the error, the larger the weight)
            self.particles[i].weight += 1e-300  # Add a small value to prevent the weight from becoming 0
        
        # Normalize the weights (normalize so that the sum of the weights of all particles is 1, as they are treated as probabilities during resampling)
        sum_weight = sum([particle.weight for particle in self.particles])
        for i in range(self.num_particles):
            self.particles[i].weight /= sum_weight

    def gaussian_probability(self, residual: float, sigma: float):
        """
        Probability density function based on Gaussian distribution
        Parameters
        ---
        residual : float
            Error
        sigma : float   
            Standard deviation
        
        Returns
        ---
        probability : float
            Probability density function based on Gaussian distribution
        """
        coefficient = 1.0 / (math.sqrt(2.0 * math.pi) * sigma)
        exponential = math.exp(-0.5 * (residual / sigma) ** 2)
        return coefficient * exponential

    def resampling(self):
        """
        Resampling
            Resample particles based on their weights
            Particles with large weights are more likely to be selected multiple times, while particles with small weights are less likely to be selected
        """
        # Preparation for resampling
        particle_indices = range(self.num_particles) # Indices of particles
        tmp_particles = self.particles # Temporary storage for particles
        weights = np.array([particle.weight for particle in self.particles]) # Weights of all particles
        rng = np.random.default_rng() # Random number generator

        # Resampling
        for i in range(self.num_particles):
            # Select particles based on their weights
            index = rng.choice(particle_indices, p=weights, replace=True) 
            self.particles[i] = copy.deepcopy(tmp_particles[index])

            # Add noise randomly
            self.particles[i].x += random.gauss(0, self.resampling_x_noise_dist)
            self.particles[i].y += random.gauss(0, self.resampling_y_noise_dist)
            self.particles[i].theta += random.gauss(0, self.resampling_theta_noise_dist)

            # Reset weight
            self.particles[i].weight = 1. / self.num_particles
            
            # For drawing
            if self.draw_particles:
                pybullet.resetBasePositionAndOrientation(self.particle_ids[i], [self.particles[i].x, self.particles[i].y, 0.0], pybullet.getQuaternionFromEuler([0, 0, self.particles[i].theta]))

    def estimate_pose(self):
        """
        Estimate the position of the actual robot
            Estimate the position of the actual robot based on the position and weight information of all particle filters
        """

        # Get the positions and weights of all particles
        particle_positions = np.array([[particle.x, particle.y, particle.theta] for particle in self.particles])
        particle_weights = np.array([particle.weight for particle in self.particles])

        # Calculate the weighted average
        estimated_position = np.average(particle_positions, axis=0, weights=particle_weights) # Calculate the weighted average
        
        # Update the estimated position
        self.x = estimated_position[0]
        self.y = estimated_position[1]
        self.theta = estimated_position[2]

    def get_pose(self):
        """
        Return the estimated position of the actual robot by the particle filter
        """
        return self.x, self.y, self.theta

# <a id='toc6_'></a>[Setting Parameters for Mobile Robots](#toc0_)

In [5]:
# Indices of the joints for the left and right wheels
RIGHT_WHEEL_IDX = 0
LEFT_WHEEL_IDX = 1

# Constants related to the wheels
WHEEL_RADIUS = 0.05 # Radius of the wheels (match the radius of the wheels in "simple_two_wheel_car.urdf")
WHEEL_THREAD = 0.325 # Distance between the wheels (match the distance between the wheels in "simple_two_wheel_car.urdf")
CONSTANT_RIGHT_WHEEL = 1.0 # Constant for the right wheel (considering the effects of friction and environment)
CONSTANT_LEFT_WHEEL = 1.0 # Constant for the left wheel (considering the effects of friction and environment)

# Create sliders for translational and rotational velocities
linear_vel_slider = pybullet.addUserDebugParameter("linear_velocity", -10, 10, 0)
angular_vel_slider = pybullet.addUserDebugParameter("angular_velocity", -10, 10, 0)

# <a id='toc7_'></a>[Setting Parameters for Sensors](#toc0_)

This time, instead of using direct sensors, we will obtain the "position of the sensor link of the mobile robot" and the "position of each obstacle" directly from the simulation. The sensor values will be treated as the distances between these two points with added noise.

In [6]:
# Position of the sensor link
SENSOR_LINK_IDX = 5
real_sensor_noise_dist = 0.01

# <a id='toc8_'></a>[Setting Up the Environment](#toc0_)
Place walls, obstacles, and the mobile robot.

(If an obstacle is generated in such a way that it collides with the mobile robot, please regenerate it.)

In [7]:
# Parameters for the walls (surrounding all sides)
wall_width = 5
wall_height = 0.5
wall_thickness = 0.1

# Parameters for obstacles
obstacle_num = 10
size_min = 0.1
size_max = 0.3

# Create walls
wall_collision_id = pybullet.createCollisionShape(pybullet.GEOM_BOX, halfExtents=[wall_width/2, wall_thickness/2, wall_height/2])
wall_visual_id = pybullet.createVisualShape(pybullet.GEOM_BOX, halfExtents=[wall_width/2, wall_thickness/2, wall_height/2], rgbaColor=[0.5,0.5,0.5,1])
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [wall_width/2, -wall_thickness/2, wall_height/2])
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [wall_width/2, wall_width+wall_thickness/2, wall_height/2])
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [-wall_thickness/2, wall_width/2, wall_height/2], pybullet.getQuaternionFromEuler([0, 0, math.pi/2]))
pybullet.createMultiBody(0, wall_collision_id, wall_visual_id, [wall_width+wall_thickness/2, wall_width/2, wall_height/2], pybullet.getQuaternionFromEuler([0, 0, math.pi/2]))

# Randomly set the positions of obstacles
obstacles = []
for i in range(obstacle_num):
    box_size = random.uniform(size_min, size_max)
    x_init = random.uniform(0+box_size, wall_width-box_size)
    y_init = random.uniform(0+box_size, wall_width-box_size)
    yaw = random.uniform(0, math.pi)
    # If the obstacle is near the center of the area surrounded by walls, reposition the obstacle
    while (wall_width/2-box_size < x_init < wall_width/2+box_size) and (wall_width/2-box_size < y_init < wall_width/2+box_size):
        x_init = random.uniform(0+box_size, wall_width-box_size)
        y_init = random.uniform(0+box_size, wall_width-box_size)
    obstacles.append([box_size, x_init, y_init, yaw])

# Place obstacles
obstacle_positions = []
for obstacle in obstacles:
    box_size = obstacle[0]
    x_init = obstacle[1]
    y_init = obstacle[2]
    yaw = obstacle[3]
    obstacle_id = pybullet.loadURDF("../urdf/simple_box.urdf", [x_init, y_init, box_size/2], pybullet.getQuaternionFromEuler([0,0,yaw]), globalScaling=box_size*2, useFixedBase=True)
    pybullet.changeVisualShape(obstacle_id, -1, rgbaColor=[1, 0, 0, 1])
    obstacle_positions.append([x_init, y_init])

# Load the robot
# Set the initial position to the center of the area surrounded by walls
car_start_x = wall_width/2
car_start_y = wall_width/2
car_start_theta = 0
car_start_position = [car_start_x, car_start_y, 0.1]  # Set initial position (x, y, z)
car_start_orientation = pybullet.getQuaternionFromEuler([0, 0, car_start_theta])  # Set initial orientation (roll, pitch, yaw)
car_id = pybullet.loadURDF("../urdf/simple_two_wheel_car.urdf", car_start_position, car_start_orientation)

# Set the camera position for GUI mode
camera_distance = wall_width
camera_yaw = 180.0 # deg
camera_pitch = -90.1 # deg
camera_target_position = [wall_width/2, wall_width/2, 0.0]
pybullet.resetDebugVisualizerCamera(camera_distance, camera_yaw, camera_pitch, camera_target_position)

# <a id='toc9_'></a>[Setting Parameters for Particles](#toc0_)
Set the parameters related to the particle filter.

Since these parameters affect the results of self-localization, try changing the values to see how the results vary.

In [8]:
# Total number of particles
num_particles = 100

# Set the range to scatter particles initially
particle_range_x = [car_start_position[0]-0.5, car_start_position[0]+0.5] # Range of ±0.5m from the initial position of the robot
particle_range_y = [car_start_position[1]-0.5, car_start_position[1]+0.5] # Range of ±0.5m from the initial position of the robot
particle_range_theta = [car_start_position[2]-math.radians(-10), car_start_position[2]+math.radians(10)] # Range of ±10° from the initial orientation of the robot

# Set the variance of noise
v_noise_dist = 0.01
omega_noise_dist = 0.01
sensor_noise_dist = 0.01

# Noise for resampling
resampling_x_noise_dist = 0.01
resampling_y_noise_dist = 0.01
resampling_theta_noise_dist = 0.001

# Set the size of particles (for drawing)
particle_size = 0.02

# <a id='toc10_'></a>[Running the Simulation](#toc0_)

When the simulation is run, self-localization using the particle filter will be performed.

In [9]:
import sys
# Initialize debug drawing
pybullet.removeAllUserDebugItems()

# Get the initial angles of the left and right wheels
initial_angle_right_wheel = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
initial_angle_left_wheel = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

# Initialize the angles of the left and right wheels
last_odom_x = car_start_x
last_odom_y = car_start_y
last_odom_theta = car_start_theta

# Instantiate the odometry class
wheel_odometry = WheelOdometry([car_start_x, car_start_y, car_start_theta], initial_angle_right_wheel, initial_angle_left_wheel, WHEEL_RADIUS, WHEEL_THREAD, CONSTANT_RIGHT_WHEEL, CONSTANT_LEFT_WHEEL)

# Instantiate the particle filter class
particle_filter = ParticleFilter([car_start_x, car_start_y, car_start_theta], num_particles, particle_range_x, particle_range_y, particle_range_theta, v_noise_dist, omega_noise_dist, sensor_noise_dist, obstacle_positions, resampling_x_noise_dist, resampling_y_noise_dist, resampling_theta_noise_dist, draw_particles=True, particle_size=particle_size)

# Reset the mobile robot to the initial position
pybullet.resetBasePositionAndOrientation(car_id, car_start_position, car_start_orientation)

while True:
    # Get the values from the sliders
    linear_velocity = pybullet.readUserDebugParameter(0)
    angular_velocity = pybullet.readUserDebugParameter(1)

    # Calculate the command velocities for the left and right wheels from the translational and rotational velocities
    right_wheel_velocity = linear_velocity - angular_velocity * WHEEL_THREAD / 2
    left_wheel_velocity = linear_velocity + angular_velocity * WHEEL_THREAD / 2

    # Set the velocities
    pybullet.setJointMotorControl2(car_id, RIGHT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=right_wheel_velocity)
    pybullet.setJointMotorControl2(car_id, LEFT_WHEEL_IDX, pybullet.VELOCITY_CONTROL, targetVelocity=left_wheel_velocity)
    pybullet.stepSimulation()

    # Get the current angles of the wheels
    current_angle_wheel_right = pybullet.getJointState(car_id, RIGHT_WHEEL_IDX)[0]
    current_angle_wheel_left = pybullet.getJointState(car_id, LEFT_WHEEL_IDX)[0]

    # Calculate odometry
    wheel_odometry.update_position(current_angle_wheel_right, current_angle_wheel_left)
    odom_x, odom_y, odom_theta = wheel_odometry.get_position()

    # Calculate velocity and angular velocity from odometry
    v = math.sqrt((odom_x - last_odom_x)**2 + (odom_y - last_odom_y)**2) / time_step
    omega = (odom_theta - last_odom_theta) / time_step

    # Update odometry
    last_odom_x = odom_x
    last_odom_y = odom_y
    last_odom_theta = odom_theta

    # Get the sensor link position
    sensor_link_position = pybullet.getLinkState(car_id, SENSOR_LINK_IDX)[0]

    # Measure the distance to all obstacles
    obstacle_distances = []
    for obstacle_position in obstacle_positions:
        # Calculate the distance between the particle and the obstacle
        #   This time, instead of actually measuring with a sensor, we directly obtain the "position of the sensor link of the mobile robot" and the "position of each obstacle" from the simulation, and treat the distance between the two points with added noise as the sensor value
        #   In reality, it is common to calculate the error between the "measurement result of the actual robot's LiDAR" and the "measurement result of the particle's (pseudo) LiDAR" and calculate the weight based on that error (this time, for simplicity, we directly obtain the sensor value)
        distance = math.sqrt((sensor_link_position[0] - obstacle_position[0])**2 + (sensor_link_position[1] - obstacle_position[1])**2) # Distance between the "position of the sensor link of the mobile robot" and the "position of the obstacle"
        distance += random.gauss(0, real_sensor_noise_dist) # Add sensor noise
        obstacle_distances.append(distance)
    
    # Update the particle filter (main process) ############################################################
    particle_filter.update_motion_model(v, omega, time_step) # Update the position of each particle based on the motion model (equivalent to position update by odometry)
    particle_filter.measurement_sensor() # Measure (pseudo) sensor values for each particle
    particle_filter.update_weight(obstacle_distances) # Update the weights of the particles
    particle_filter.resampling() # Resampling
    particle_filter.estimate_pose() # Estimate the position of the particles
    #############################################################################################################
    estimated_pose = particle_filter.get_pose() # Get the estimated position by the particle filter

    # Display the estimated position and the true position
    true_pose = [pybullet.getBasePositionAndOrientation(car_id)[0][0], pybullet.getBasePositionAndOrientation(car_id)[0][1], pybullet.getEulerFromQuaternion(pybullet.getBasePositionAndOrientation(car_id)[1])[2]]
    sys.stdout.write(f"\r【estimated position: ({estimated_pose[0]:.2f}, {estimated_pose[1]:.2f}, {estimated_pose[2]:.2f})】【true position: ({true_pose[0]:.2f}, {true_pose[1]:.2f}, {true_pose[2]:.2f})】               ")

    time.sleep(time_step)

【estimated position: (2.73, 2.52, 0.28)】【true position: (2.73, 2.50, 0.02)】               